## Model serving with HSML

In this example, we are going to serve the model that we created in the model training notebook.

For the example to work, you need to have serving enabled in your project. In the settings tab for your project, select Serving to enable it. Now your UI should show a new tab called Model Serving.

A model deployment (also called "model serving") can be created directly in the Hopsworks UI, by clicking on Model Serving and then on Create New Serving. In this example, however, we will create it through code with the HSML library.

![tutorial-flow](images/end_to_end.png)

### About Model Serving

Models can be served via KFServing or "default" serving, which means a Docker container exposing a Flask server. For KFServing models, or models written in Tensorflow, you do not need to write a prediction file (see the section below). However, for sklearn models using default serving, you do need to proceed to write a prediction file.

In order to use KFServing, you must have Kubernetes installed and enabled on your cluster.

### Create the Prediction File

In order to deploy a model, you need to write a Python file containing the logic to return a prediction from the model. Don't worry, this is usually a matter of just modifying some paths in a template script. An example can be seen in the code block below, where we have taken [this](https://hopsworks.readthedocs.io/en/latest/hopsml/python_model_serving.html#serving-python-based-models-on-hopsworks) Scikit-learn template script and changed two paths (see comments). 

In [ ]:
from sklearn.externals import joblib
from hops import hdfs # TODO this library should not be needed.

class Predict(object):

    def __init__(self):
        """ Initializes the serving state, reads a trained model from HDFS"""
        self.model_path = "Models/fraud_tutorial_model/1/model.pkl" # Changed to our path.
        print("Copying Scikit-Learn model from HDFS to local directory")
        hdfs.copy_to_local(self.model_path)
        print("Reading local Scikit-Learn model for serving")
        self.model = joblib.load("./model.pkl") # Changed to our path.
        print("Initialization Complete")


    def predict(self, inputs):
        """ Serves a prediction request usign a trained model"""
        return self.model.predict(inputs).tolist() # Numpy Arrays are not JSON serializable

If you wonder why we use the path `Models/fraud_tutorial_model/1/model.pkl`, it is useful to know that the Data Sets tab in the Hopsworks UI lets you browse among the different files in the project. Registered models will be found underneath the `Models` directory. Since we saved our model with the name `fraud_tutorial`, that's the directory we should look in. `1` is just the version of the model we want to deploy.

This script needs to be put into a known location in the Hopsworks file system. Let's call the file `predict_example.py` and put it in the `Models` directory. The following should be the path to the script.

In [ ]:
PREDICTOR_SCRIPT_PATH = "/Projects/fraud_tutorial/Models/predict_example.py"

## Create the deployment

Here, we fetch the model we want from the model registry and define a configuration for the deployment. For the configuration, we need to specify the serving type (default or KFserving) and in this case, since we use default serving and an sklearn model, we need to give the location of the prediction script.

In [ ]:
import hsml
from hsml.predictor_config import PredictorConfig

conn = hsml.connection()
mr = conn.get_model_registry()

# Use the model name from the previous notebook.
model = mr.get_model("fraud_tutorial_model", version=1)

predictor_config = PredictorConfig(
    model_server="PYTHON",
    serving_tool="DEFAULT",
    script_file=PREDICTOR_SCRIPT_PATH,
)

# Give it any name you want
deployment = model.deploy(name="frauddeployment", predictor_config=predictor_config)

The deployment has now been registered. However, to start it you need to run:

In [ ]:
# TODO These two lines should not be necessary (?) once the ID error has been resolved.
ms = conn.get_model_serving()
deployment = ms.get_deployment("frauddeployment")

deployment.start()

### Using the deployment

Let's use the input example that we registered together with the model to query the deployment.

In [33]:
test_inputs = [model.input_example]
print(test_inputs)

data = {
    "inputs": test_inputs
}

deployment.predict(data)

### Use REST endpoint

You can also use a REST endpoint for your model. To do this you need to create an API key with 'serving' enabled, and retrieve the endpoint URL from the Model Serving UI.

(**NOTE currently this has to be done in the old UI**)

Go to the Model Serving UI and click on the eye icon next to a model to retrieve the endpoint URL. The shorter URL is an internal endpoint that you can only reach from within Hopsworks. If you want to call it from outside, you need one of the longer URLs. Make sure to use https instead of http. (**TODO this should be fixed**)


In [7]:
import requests

API_KEY = ""  # Put your API key here.

data = {"inputs": test_inputs}
url = ""  # Put the URL you found in the UI here.
headers = {
    "Content-Type": "application/json", "Accept": "application/json",
    "Authorization": f"ApiKey {API_KEY}"}

response = requests.post(url, headers=headers, json=data)
response.json()

### Stop Deployment

To stop the deployment we simply run:

In [ ]:
deployment.stop()

### Next Steps

In the next notebook we'll take a look at how to automate jobs in Hopsworks using Airflow.